In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import datetime as dt
import pytz
from typing import List, Dict, Tuple

# VERBOSE: print(f"Python version: {sys.version}")

# Configuration
TIMEZONE = pytz.timezone("Africa/Dar_es_Salaam")

# Dataset configuration - using simple relative paths
DATA_FOLDER = Path("data/omnisense")
EXTERNAL_TEMP_FILE = DATA_FOLDER / "open-meteo-7.07S39.30E81m.csv"

USECOLS = [1, 2, 3]  # B, C, D columns

# Logger name mapping
LOGGER_NAMES = {
    "861011": "External Ambient (861011)",
    "780981": "Living Room (780981)",
    "639148": "Study (639148)",
    "759522": "Bed 1 (759522)",
    "759521": "Bed 2 (759521)",
    "759209": "Bed 3 (759209)",
    "759492": "Bed 4 (759492)",
    "861968": "Living Room (below metal) (861968)",
    "759493": "Living Room (above ceiling) (759493)",
    "759498": "Dauda's House (759498)",
    "861004": "Bed 3 (above ceiling) (861004)",
    "861034": "Bed 3 (above ceiling) (861034)",
    "759519": "Bed 4 (below metal) (759519)",
    "759489": "Bed 4 (above ceiling) (759489)",
    "govee": "Govee Smart Hygrometer",
    "External (Open-Meteo)": "External Temperature"
}

def get_season_lines(start_date, end_date):
    """Generate vertical lines for season boundaries."""
    lines = []
    start_year = start_date.year
    end_year = end_date.year
    
    # Season dates: 01/06, 01/11, 01/01, 01/03 (dd/mm) with names
    season_info = [
        (6, 1, "June Dry Season (Kiangazi)"),
        (11, 1, "Short Rains (Vuli)"),
        (1, 1, "January Dry Season (Kiangazi)"),
        (3, 1, "Long Rains (Masika)")
    ]
    
    for year in range(start_year - 1, end_year + 2):  # Extended range to ensure coverage
        for month, day, name in season_info:
            season_timestamp = pd.Timestamp(year=year, month=month, day=day, tz=TIMEZONE)
            if start_date <= season_timestamp <= end_date:
                lines.append((season_timestamp, name))
    
    return sorted(lines, key=lambda x: x[0])


def get_season_for_date(date):
    """Get the season name for a given date."""
    month = date.month
    day = date.day
    
    # Define season boundaries (month, day)
    if (month == 6 and day >= 1) or (month > 6 and month < 11):
        return "June Dry Season (Kiangazi)"
    elif (month == 11 and day >= 1) or (month == 12):
        return "Short Rains (Vuli)"
    elif (month == 1 and day >= 1) or (month == 2):
        return "January Dry Season (Kiangazi)"
    elif (month == 3 and day >= 1) or (month < 6):
        return "Long Rains (Masika)"
    else:
        return "Short Rains (Vuli)"  # Default fallback

Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]


In [2]:
def load_external_temperature() -> pd.DataFrame:
    """Load external temperature data from Open-Meteo CSV."""
    try:
        if not EXTERNAL_TEMP_FILE.exists():
            # VERBOSE: print(f"Warning: External temperature file not found at {EXTERNAL_TEMP_FILE}")
            return pd.DataFrame()
        
        # Read CSV, skipping first 3 metadata rows
        df = pd.read_csv(EXTERNAL_TEMP_FILE, skiprows=3)
        
        # Rename columns to match our schema
        df = df.rename(columns={
            'time': 'datetime',
            'temperature_2m (°C)': 'temperature',
            'relative_humidity_2m (%)': 'humidity'
        })
        
        # Add logger_id
        df['logger_id'] = 'External (Open-Meteo)'
        
        # Convert datetime
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
        df['temperature'] = pd.to_numeric(df['temperature'], errors='coerce')
        df['humidity'] = pd.to_numeric(df['humidity'], errors='coerce')
        
        # Drop any invalid rows
        df = df.dropna(subset=['datetime', 'temperature', 'humidity'])
        
        # VERBOSE: print(f"Loaded {len(df)} external temperature records from Open-Meteo")
        # VERBOSE: print(f"  Date range: {df['datetime'].min()} to {df['datetime'].max()}")
        
        return df[['datetime', 'temperature', 'humidity', 'logger_id']]
    
    except Exception as e:
        # VERBOSE: print(f"Error loading external temperature: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()


def load_omnisense_csv(path: Path) -> pd.DataFrame:
    """Load and parse Omnisense CSV file with multiple datasets."""
    try:
        with open(path, 'r') as f:
            lines = f.readlines()

        all_dfs = []
        i = 0

        while i < len(lines):
            # Look for dataset header
            if 'sensor_desc,site_name' in lines[i]:
                # Get sensor description (next line) - everything except the site name at the end
                desc_line = lines[i + 1].strip()
                # Remove the site name (e.g., "ARC CEV Tanzania") - it's after the last comma
                parts = desc_line.split(',')
                sensor_desc = ','.join(parts[:-1]) if len(parts) > 1 else parts[0]
                sensor_desc = sensor_desc.strip()

                # Get column headers (line after description)
                col_headers = lines[i + 2].strip().split(',')

                # Skip weather station data
                if 'Weather Station' in sensor_desc:
                    # VERBOSE: print(f"Skipping {sensor_desc} (weather station)")
                    i += 1
                    continue

                # Check if this dataset has temperature and humidity
                if 'temperature' in col_headers and 'humidity' in col_headers:
                    # Find column indices
                    temp_idx = col_headers.index('temperature')
                    humidity_idx = col_headers.index('humidity')

                    # Find datetime column (could be 'read_date' or similar)
                    date_col = None
                    for col in ['read_date', 'datetime', 'date', 'time']:
                        if col in col_headers:
                            date_col = col
                            date_idx = col_headers.index(col)
                            break

                    if date_col is None:
                        # VERBOSE: print(f"Warning: No date column found for {sensor_desc}, skipping")
                        i += 1
                        continue

                    # Find where this dataset ends (next sensor_desc or end of file)
                    data_start = i + 3
                    data_end = data_start
                    for j in range(data_start, len(lines)):
                        if 'sensor_desc,site_name' in lines[j]:
                            data_end = j
                            break
                        data_end = j + 1

                    # Parse data rows
                    data_rows = []
                    for row_line in lines[data_start:data_end]:
                        row = row_line.strip().split(',')
                        if len(row) > max(temp_idx, humidity_idx, date_idx):
                            try:
                                data_rows.append({
                                    'datetime': row[date_idx],
                                    'temperature': row[temp_idx],
                                    'humidity': row[humidity_idx]
                                })
                            except (IndexError, ValueError):
                                continue

                    if data_rows:
                        # Create DataFrame for this sensor
                        df = pd.DataFrame(data_rows)
                        df['logger_id'] = sensor_desc
                        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
                        df['temperature'] = pd.to_numeric(df['temperature'], errors='coerce')
                        df['humidity'] = pd.to_numeric(df['humidity'], errors='coerce')
                        df = df.dropna()

                        if not df.empty:
                            all_dfs.append(df)
                            # VERBOSE: print(f"Loaded {len(df)} records from: {sensor_desc}")

                    i = data_end
                else:
                    # Skip datasets without temperature and humidity
                    # VERBOSE: print(f"Skipping {sensor_desc} (no temperature/humidity data)")
                    i += 1
            else:
                i += 1

        if not all_dfs:
            return pd.DataFrame()

        # Combine all sensor data
        return pd.concat(all_dfs, ignore_index=True)

    except Exception as e:
        # VERBOSE: print(f"Error loading {path.name}: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()


def load_all_data() -> pd.DataFrame:
    """Load and combine all CSV files from the data folder, plus external temperature."""
    # Load omnisense sensor data
    csv_files = sorted(p for p in DATA_FOLDER.glob("*.csv")
                      if not p.name.startswith("~$") and p.name != EXTERNAL_TEMP_FILE.name)

    # VERBOSE: print(f"Found {len(csv_files)} Omnisense CSV file(s) in {DATA_FOLDER}")

    dfs = []
    for csv_file in csv_files:
        # VERBOSE: print(f"\nProcessing {csv_file.name}...")
        df = load_omnisense_csv(csv_file)
        if not df.empty:
            dfs.append(df)

    # Load external temperature data
    # VERBOSE: print(f"\nLoading external temperature data...")
    external_df = load_external_temperature()
    if not external_df.empty:
        dfs.append(external_df)

    if not dfs:
        raise ValueError(f"No valid data found in {DATA_FOLDER}")

    # VERBOSE: print(f"\nTotal datasets loaded: {len(dfs)}")

    # Combine all data
    df_all = pd.concat(dfs, ignore_index=True).sort_values("datetime")

    # VERBOSE: print(f"Total records: {len(df_all)}")
    # VERBOSE: print(f"Unique loggers: {df_all['logger_id'].nunique()}")
    # VERBOSE: print(f"Logger IDs: {sorted(df_all['logger_id'].unique())}")

    # Ensure timezone-aware datetimes
    df_all["datetime"] = (
        pd.to_datetime(df_all["datetime"], errors="coerce")
        .dt.tz_localize(TIMEZONE, nonexistent="shift_forward", ambiguous="NaT")
    )

    df_all = df_all.dropna(subset=["datetime"]).set_index("datetime").sort_index()

    # Precompute ISO calendar
    iso = df_all.index.isocalendar()
    df_all["iso_year"] = iso.year
    df_all["iso_week"] = iso.week

    return df_all

In [3]:
# Load data
df_all = load_all_data()

Found 1 Omnisense CSV file(s) in data/omnisense

Processing 070226.csv...
Loaded 2501 records from: House 5, Kitchen
Loaded 2416 records from: House 5, Bed 4
Loaded 2498 records from: House 5, Bed 4, above ceiling
Loaded 2500 records from: House 5, Bed 2
Loaded 2499 records from: House 5, Living Room
Loaded 2498 records from: House 5, Mother's Bedroom
Loaded 2499 records from: House 5, Washrooms area
Loaded 2500 records from: House 3, Bed 2
Loaded 2498 records from: House 5, Bed 3
Skipping Sun, Wind, Rain weather station gateway (in external box) (no temperature/humidity data)
Skipping Weather Station, T & RH (weather station)
Loaded 4583 records from: House 5, Metal Roof, above Bed 4
Skipping Performance stats (no temperature/humidity data)

Loading external temperature data...
Loaded 504 external temperature records from Open-Meteo
  Date range: 2026-01-17 00:00:00 to 2026-02-06 23:00:00

Total datasets loaded: 2
Total records: 27496
Unique loggers: 11
Logger IDs: ['External (Open-Me

In [4]:
def create_colour_map(loggers: List[str]) -> Dict[str, str]:
    """Create a colour mapping for loggers with normal, distinct colors."""
    # Standard color palette with good contrast
    colors = [
        "#1f77b4",  # Blue
        "#ff7f0e",  # Orange  
        "#2ca02c",  # Green
        "#d62728",  # Red
        "#9467bd",  # Purple
        "#8c564b",  # Brown
        "#e377c2",  # Pink
        "#7f7f7f",  # Gray
        "#bcbd22",  # Olive
        "#17becf",  # Cyan
        "#aec7e8",  # Light Blue
        "#ffbb78",  # Light Orange
        "#98df8a",  # Light Green
        "#ff9896",  # Light Red
        "#c5b0d5",  # Light Purple
        "#c49c94",  # Light Brown
        "#f7b6d3",  # Light Pink
        "#c7c7c7",  # Light Gray
        "#dbdb8d",  # Light Olive
        "#9edae5",  # Light Cyan
        "#393b79",  # Dark Blue
        "#637939",  # Dark Green
        "#8c6d31",  # Dark Orange
        "#843c39"   # Dark Red
    ]
    return {logger: colors[i % len(colors)] for i, logger in enumerate(loggers)}


def create_checkboxes(items: List[str], default: bool = True) -> Dict[str, widgets.Checkbox]:
    """Create a dictionary of checkboxes for items."""
    return {
        item: widgets.Checkbox(
            value=default,
            description=str(item),
            indent=False,
            layout=widgets.Layout(height="18px")
        )
        for item in items
    }


def create_time_selectors(df: pd.DataFrame) -> Tuple:
    """Create all time selection widgets."""
    date_min, date_max = df.index.min(), df.index.max()
    
    # Time mode dropdown
    time_mode = widgets.Dropdown(
        options=["All time", "Between dates", "Year", "Month", "Week", "Day"],
        value="All time",
        description="Range:",
        layout=widgets.Layout(width="200px")
    )
    
    # Between dates selector
    between_box = widgets.HBox([
        widgets.DatetimePicker(description="Start", value=date_min, step=60),
        widgets.DatetimePicker(description="End", value=date_max, step=60)
    ])
    
    # Year selector
    available_years = sorted(df.index.year.unique())
    year_select = widgets.Dropdown(
        options=available_years, 
        description="Year:", 
        layout=widgets.Layout(display="none")
    )
    
    # Month selector
    available_months = sorted({(y, m) for y, m in zip(df.index.year, df.index.month)})
    month_select = widgets.Dropdown(
        options=[(f"{dt.date(y, m, 1):%B %Y}", (y, m)) for y, m in available_months],
        description="Month:",
        layout=widgets.Layout(display="none")
    )
    
    # Week selector
    available_weeks = sorted({(y, int(w)) for y, w in zip(df["iso_year"], df["iso_week"])})
    week_select = widgets.Dropdown(
        options=[(f"Week {w}, {y}", (y, w)) for y, w in available_weeks],
        description="Week:",
        layout=widgets.Layout(display="none")
    )
    
    # Day selector
    available_days = sorted(df.index.normalize().unique())
    day_select = widgets.Dropdown(
        options=[(d.strftime("%d %b %Y"), d) for d in available_days],
        description="Day:",
        layout=widgets.Layout(display="none")
    )
    
    return time_mode, between_box, year_select, month_select, week_select, day_select


def create_gap_broken_traces(df: pd.DataFrame, selected_metrics: list, logger_display_names: dict, colour_map: dict, max_gap_hours: float = 12.0):
    """Create separate traces for each logger, breaking on large time gaps."""
    traces = []
    
    if df.empty:
        return traces
    
    gap_threshold = pd.Timedelta(hours=max_gap_hours)
    
    # Group by logger_id to handle each logger separately
    for logger_id in df['logger_id'].unique():
        logger_data = df[df['logger_id'] == logger_id].copy().sort_index()
        
        if logger_data.empty:
            continue
            
        logger_display_name = logger_display_names.get(logger_id, logger_id)
        logger_color = colour_map.get(logger_id, "#1f77b4")
        
        # Find gap locations
        if len(logger_data) < 2:
            # Single point or empty - create one trace
            gap_indices = []
        else:
            time_diffs = logger_data.index.to_series().diff()
            large_gaps = time_diffs > gap_threshold
            gap_indices = logger_data.index[large_gaps].tolist()
        
        # Split data at gap locations
        start_idx = 0
        logger_indices = logger_data.index.tolist()
        
        sub_trace_count = 0
        first_trace_per_logger = False  # Track if we've shown legend for this logger yet
        
        for gap_idx in gap_indices + [None]:  # Add None to handle the last segment
            if gap_idx is not None:
                end_idx = logger_indices.index(gap_idx)
            else:
                end_idx = len(logger_indices)
            
            if start_idx < end_idx:
                # Create sub-trace for this segment
                segment_data = logger_data.iloc[start_idx:end_idx]
                
                if not segment_data.empty:
                    for metric in selected_metrics:
                        if metric in segment_data.columns:
                            # Ensure no None values in x or name
                            x_values = segment_data.index.tolist()
                            y_values = segment_data[metric].tolist()
                            
                            # Filter out any None/NaN values
                            valid_pairs = [(x, y) for x, y in zip(x_values, y_values) 
                                         if x is not None and pd.notna(y)]
                            
                            if valid_pairs:
                                x_clean, y_clean = zip(*valid_pairs)
                                
                                # Only show in legend for the very first trace of this logger
                                show_legend = not first_trace_per_logger
                                if show_legend:
                                    first_trace_per_logger = True
                                
                                # Determine units for hover template
                                if metric == "temperature":
                                    unit = "°C"
                                else:  # humidity
                                    unit = "%RH"
                                
                                trace = go.Scatter(
                                    x=list(x_clean),
                                    y=list(y_clean),
                                    mode='lines',
                                    name=logger_display_name,
                                    line=dict(color=logger_color),
                                    showlegend=show_legend,
                                    connectgaps=False,
                                    legendgroup=logger_display_name,  # Group all sub-traces together
                                    hovertemplate=f"{logger_display_name}<br>%{{x|%d/%m/%Y %H:%M}}<br>{metric.title()}: %{{y:.2f}}{unit}<extra></extra>"
                                )
                                traces.append(trace)
                    
                    sub_trace_count += 1
            
            start_idx = end_idx
    
    return traces


def get_time_mask(df: pd.DataFrame, mode: str, selectors: Dict) -> pd.Series:
    """Generate boolean mask based on time selection mode."""
    selected_loggers = selectors["selected_loggers"]
    logger_mask = df["logger_id"].isin(selected_loggers)
    
    if mode == "All time":
        return logger_mask
    
    elif mode == "Between dates":
        start = pd.Timestamp(selectors["between_box"].children[0].value).tz_convert(TIMEZONE)
        end = pd.Timestamp(selectors["between_box"].children[1].value).tz_convert(TIMEZONE)
        return (df.index >= start) & (df.index <= end) & logger_mask
    
    elif mode == "Year":
        year = int(selectors["year_select"].value)
        return (df.index.year == year) & logger_mask
    
    elif mode == "Month":
        year, month = selectors["month_select"].value
        return (df.index.year == year) & (df.index.month == month) & logger_mask
    
    elif mode == "Week":
        year, week = selectors["week_select"].value
        return (df["iso_year"] == year) & (df["iso_week"] == week) & logger_mask
    
    elif mode == "Day":
        day = selectors["day_select"].value.tz_convert(TIMEZONE)
        return (df.index.normalize() == day.normalize()) & logger_mask
    
    return logger_mask

In [ ]:
def prepare_adaptive_comfort_data(df: pd.DataFrame, time_mask: pd.Series, external_logger: str, room_loggers: list) -> pd.DataFrame:
    """Prepare data for adaptive comfort chart using 7-day rolling mean of DAILY external temperature."""
    # Filter for time range first
    df_filtered = df[time_mask].copy()
    
    # Get external temperature data
    if external_logger and external_logger in df_filtered["logger_id"].values:
        external_data = df_filtered[df_filtered["logger_id"] == external_logger][["temperature"]].copy()
    else:
        # If no external logger specified, return empty
        print("Warning: No external temperature data available for adaptive comfort calculation")
        return pd.DataFrame()
    
    external_data.columns = ["external_temp"]
    
    # Step 1: Calculate DAILY mean external temperature
    daily_means = external_data.resample("d").mean()
    
    # Step 2: Calculate 7-day rolling mean of the daily means
    # This uses the previous 7 days including today
    daily_means["external_temp_7day_mean"] = daily_means["external_temp"].rolling(
        window=7, 
        min_periods=1
    ).mean()
    
    # Step 3: Upsample back to original frequency (forward-fill the daily values)
    # This assigns each hour/minute the same daily rolling mean value
    external_with_running_mean = daily_means[["external_temp_7day_mean"]].resample("h").ffill()
    
    # Step 4: Get room data
    room_data = df_filtered[df_filtered["logger_id"].isin(room_loggers)].copy()
    
    # Step 5: Merge room data with 7-day rolling mean external temperature
    # Use merge_asof to match each room measurement with the closest external temperature
    room_data = room_data.reset_index().sort_values("datetime")
    external_reset = external_with_running_mean.reset_index().sort_values("datetime")
    
    room_data = pd.merge_asof(
        room_data,
        external_reset.rename(columns={"datetime": "datetime"}),
        on="datetime",
        direction="nearest"
    )
    
    # Set datetime back as index
    room_data = room_data.set_index("datetime")
    
    # Rename column to match expected name in other functions
    room_data = room_data.rename(columns={"external_temp_7day_mean": "external_temp"})
    
    # Drop rows with missing external temperature
    room_data = room_data.dropna(subset=["external_temp"])
    
    return room_data

In [ ]:
class LoggerDataViewer:
    """Interactive logger data viewer with filtering capabilities."""
    
    def __init__(self):
        self.df = load_all_data()
        self._setup_config()
        self._create_all_widgets()
        self._create_layout()
        self._setup_observers()
    
    def _setup_config(self):
        """Setup configuration based on loaded data."""
        # Determine loggers from actual data
        unique_loggers_in_data = sorted(self.df["logger_id"].unique())
        self.unique_loggers = unique_loggers_in_data
        
        # Identify external temperature logger
        if "External (Open-Meteo)" in unique_loggers_in_data:
            self.external_logger = "External (Open-Meteo)"
            # Room loggers are all except external
            self.room_loggers = [l for l in unique_loggers_in_data if l != "External (Open-Meteo)"]
        else:
            # Fallback if external not found
            self.external_logger = None
            self.room_loggers = unique_loggers_in_data
            # VERBOSE: print("Warning: External temperature logger not found!")
        
        self.colour_map = create_colour_map(self.unique_loggers)
        
        # Map logger IDs to display names
        self.logger_display_names = {
            logger_id: LOGGER_NAMES.get(logger_id, logger_id) 
            for logger_id in self.unique_loggers
        }
        
        # VERBOSE: print(f"Found loggers: {self.unique_loggers}")
        # VERBOSE: print(f"External logger: {self.external_logger}")
        # VERBOSE: print(f"Room loggers: {self.room_loggers}")

In [ ]:
# Store reference to display function for use in class methods
_ipython_display = display

In [ ]:
# Widget creation methods
def _create_all_widgets(self):
    """Create all widgets based on current configuration."""
    # Chart type selector
    self.chart_type = widgets.Dropdown(
        options=["Line Graph", "Adaptive Comfort"],
        value="Line Graph",
        description="Chart Type:",
        layout=widgets.Layout(width="200px")
    )
    
    # Comfort model selector for adaptive comfort
    self.boundary_model = widgets.Dropdown(
        options=[
            ("Default model", "default"),
            ("RH>60% (Vellei et al.)", "rh_gt_60"),
            ("40%<RH≤60% (Vellei et al.)", "rh_40_60"),
            ("RH≤40% (Vellei et al.)", "rh_le_40"),
            ("None", "none"),
        ],
        value="default",
        description="Comfort:",
        layout=widgets.Layout(width="320px")
    )
    
    # Create widgets
    self.logger_checkboxes = create_checkboxes([
        self.logger_display_names[logger_id] for logger_id in self.unique_loggers
    ])
    self.metric_checkboxes = create_checkboxes(["Temperature", "Humidity"])
    
    # Create room logger checkboxes for adaptive comfort chart
    self.room_logger_checkboxes = create_checkboxes([
        self.logger_display_names[logger_id] for logger_id in self.room_loggers if logger_id in self.unique_loggers
    ])
    
    # Create comfort percentage display
    self.comfort_stats_out = widgets.Output()
    
    # Add 32°C threshold checkbox
    self.threshold_checkbox = widgets.Checkbox(
        value=True,
        description="32°C Threshold",
        indent=False,
        layout=widgets.Layout(height="18px")
    )
    
    # Add season lines checkbox
    self.season_lines_checkbox = widgets.Checkbox(
        value=True,
        description="Season Lines",
        indent=False,
        layout=widgets.Layout(height="18px")
    )
    
    # Add download buttons
    self.download_button = widgets.Button(
        description="Download Chart",
        button_style="success",
        layout=widgets.Layout(width="150px")
    )
    
    self.download_with_stats_button = widgets.Button(
        description="Download with Stats",
        button_style="info",
        layout=widgets.Layout(width="150px")
    )
    
    (self.time_mode, self.between_box, self.year_select, 
     self.month_select, self.week_select, self.day_select) = create_time_selectors(self.df)
    
    self.plot_out = widgets.Output()
    self.loading_out = widgets.Output(layout=widgets.Layout(height="30px"))
    # Debug output removed for Voilà compatibility

def _create_layout(self):
    """Create the widget layout."""
    logger_box = widgets.VBox(
        [widgets.HTML("<b>Loggers</b>")] + list(self.logger_checkboxes.values()) + [self.threshold_checkbox, self.season_lines_checkbox],
        layout=widgets.Layout(width="300px")
    )
    
    metric_box = widgets.VBox(
        [widgets.HTML("<b>Metrics</b>")] + list(self.metric_checkboxes.values()),
        layout=widgets.Layout(width="150px")
    )
    
    room_logger_box = widgets.VBox(
        [widgets.HTML("<b>Room Loggers</b>")] + list(self.room_logger_checkboxes.values()),
        layout=widgets.Layout(width="300px")
    )
    
    # Comfort stats box for adaptive comfort
    comfort_stats_box = widgets.VBox([
        widgets.HTML("<b>Comfort Statistics</b>"),
        self.comfort_stats_out
    ], layout=widgets.Layout(width="400px", margin="0px 0px 20px 0px"))
    
    download_box = widgets.VBox([
        widgets.HTML("<b>Export</b>"),
        self.download_button,
        self.download_with_stats_button,
        self.loading_out
    ], layout=widgets.Layout(width="150px"))
    
    self.time_controls = widgets.VBox([
        self.chart_type,
        self.boundary_model,
        self.time_mode, 
        self.between_box, 
        self.year_select, 
        self.month_select, 
        self.week_select, 
        self.day_select
    ])
    
    self.logger_box = logger_box
    self.metric_box = metric_box
    self.room_logger_box = room_logger_box
    self.comfort_stats_box = comfort_stats_box
    self.controls = widgets.HBox([logger_box, metric_box, room_logger_box, comfort_stats_box, download_box])
    
    # Initial visibility
    self._update_controls_visibility()

LoggerDataViewer._create_all_widgets = _create_all_widgets
LoggerDataViewer._create_layout = _create_layout

In [ ]:
# Observer and event handler methods
def _setup_observers(self):
    """Set up all widget observers."""
    # Time mode visibility
    self.time_mode.observe(self._update_visibility, "value")
    
    # Chart type changes
    self.chart_type.observe(self._on_chart_type_change, "value")
    
    # Comfort model changes
    self.boundary_model.observe(self._on_widget_change, "value")
    
    # Plot updates
    for cb in list(self.logger_checkboxes.values()) + list(self.metric_checkboxes.values()) + list(self.room_logger_checkboxes.values()):
        cb.observe(self._on_checkbox_change, "value")
    
    self.threshold_checkbox.observe(self._on_checkbox_change, "value")
    self.season_lines_checkbox.observe(self._on_checkbox_change, "value")
    self.download_button.on_click(self._download_chart)
    self.download_with_stats_button.on_click(self._download_chart_with_stats)
    
    for widget in [self.between_box.children[0], self.between_box.children[1],
                  self.year_select, self.month_select, self.week_select, self.day_select]:
        widget.observe(self._on_time_widget_change, "value")

def _on_chart_type_change(self, change):
    """Handler for chart type changes."""
    self._debug_log(f"*** CHART TYPE CHANGE: {change['new']} ***")
    self._update_controls_visibility()
    self.update_plot()

def _on_checkbox_change(self, change):
    """Handler for checkbox changes."""
    self._debug_log(f"*** CHECKBOX CHANGE: {change['owner'].description} = {change['new']} ***")
    self.update_plot()

def _on_widget_change(self, change):
    """Handler for widget changes."""
    self._debug_log(f"*** WIDGET CHANGE: {change['owner'].description} = {change['new']} ***")
    self.update_plot()

def _on_time_widget_change(self, change):
    """Handler for time widget changes with loading indicator."""
    self._debug_log(f"*** TIME WIDGET CHANGE: {change['owner'].description} = {change['new']} ***")
    self.update_plot()

def _update_controls_visibility(self):
    """Update visibility of controls based on chart type."""
    is_line_graph = self.chart_type.value == "Line Graph"
    
    # Show/hide logger and metric boxes
    self.logger_box.layout.display = "flex" if is_line_graph else "none"
    self.metric_box.layout.display = "flex" if is_line_graph else "none"
    
    # Show/hide room logger box for adaptive comfort
    self.room_logger_box.layout.display = "none" if is_line_graph else "flex"
    
    # Show/hide comfort stats box for adaptive comfort
    self.comfort_stats_box.layout.display = "none" if is_line_graph else "flex"
    
    # Show/hide download with stats button for adaptive comfort
    self.download_with_stats_button.layout.display = "none" if is_line_graph else "flex"
    
    # Show comfort model selector only for adaptive comfort
    self.boundary_model.layout.display = "none" if is_line_graph else "flex"

def _update_visibility(self, change):
    """Update visibility of time selectors based on mode."""
    self._debug_log(f"*** TIME MODE VISIBILITY CHANGE: {change['new']} ***")
    
    # Hide all
    for widget in [self.between_box, self.year_select, self.month_select, 
                  self.week_select, self.day_select]:
        widget.layout.display = "none"
    
    # Show relevant selector
    mode = change["new"]
    visibility_map = {
        "Between dates": self.between_box,
        "Year": self.year_select,
        "Month": self.month_select,
        "Week": self.week_select,
        "Day": self.day_select
    }
    
    if mode in visibility_map:
        visibility_map[mode].layout.display = "flex"
    
    # Update plot when time mode changes
    self.update_plot()

def _show_loading(self):
    """Show loading indicator."""
    with self.loading_out:
        # VERBOSE: print("Loading...")
        pass

def _hide_loading(self):
    """Hide loading indicator."""
    self.loading_out.clear_output()

def _debug_log(self, message):
    """Debug logging disabled for Voilà."""
    pass

LoggerDataViewer._setup_observers = _setup_observers
LoggerDataViewer._on_chart_type_change = _on_chart_type_change
LoggerDataViewer._on_checkbox_change = _on_checkbox_change
LoggerDataViewer._on_widget_change = _on_widget_change
LoggerDataViewer._on_time_widget_change = _on_time_widget_change
LoggerDataViewer._update_controls_visibility = _update_controls_visibility
LoggerDataViewer._update_visibility = _update_visibility
LoggerDataViewer._show_loading = _show_loading
LoggerDataViewer._hide_loading = _hide_loading
LoggerDataViewer._debug_log = _debug_log

In [ ]:
# Main plot update method
def update_plot(self):
    """Update the plot based on current selections."""
    self._debug_log("=== Starting update_plot() ===")
    self._debug_log(f"Chart type: {self.chart_type.value}")
    self._debug_log(f"Time mode: {self.time_mode.value}")
    
    self._show_loading()
    self.plot_out.clear_output(wait=True)
    
    try:
        with self.plot_out:
            self._debug_log("Generating new plot...")
            if self.chart_type.value == "Line Graph":
                fig = self._plot_line_graph()
            else:
                fig = self._plot_adaptive_comfort()
            
            if fig is not None:
                self._debug_log("Plot generated successfully, updating current_fig")
                self.current_fig = fig
                self._debug_log("Displaying new plot")
                fig.show(config={
                    'displayModeBar': True,
                    'modeBarButtonsToRemove': [
                        'zoom2d', 'pan2d', 'select2d', 'lasso2d', 'zoomIn2d', 'zoomOut2d',
                        'resetScale2d', 'toImage', 'sendDataToCloud', 'hoverClosestCartesian',
                        'hoverCompareCartesian', 'toggleHover', 'toggleSpikelines'
                    ]
                })
                self._debug_log("Plot displayed successfully")
            else:
                self._debug_log("Plot generation returned None - no plot to display")
                # Create and display empty placeholder figure
                empty_fig = go.Figure()
                empty_fig.add_annotation(
                    text="No data available for current selection",
                    xref="paper", yref="paper",
                    x=0.5, y=0.5, xanchor='center', yanchor='middle',
                    font=dict(size=16, color="gray")
                )
                empty_fig.update_layout(
                    height=400, width=800,
                    template="plotly_white",
                    showlegend=False,
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False)
                )
                self.current_fig = empty_fig
                empty_fig.show(config={
                    'displayModeBar': True,
                    'modeBarButtonsToRemove': [
                        'zoom2d', 'pan2d', 'select2d', 'lasso2d', 'zoomIn2d', 'zoomOut2d',
                        'resetScale2d', 'toImage', 'sendDataToCloud', 'hoverClosestCartesian',
                        'hoverCompareCartesian', 'toggleHover', 'toggleSpikelines'
                    ]
                })
                self._debug_log("Displayed empty placeholder figure")
    except Exception as e:
        import traceback
        error_msg = f"Error in update_plot: {e}\n{traceback.format_exc()}"
        self._debug_log(error_msg)
        # VERBOSE: print(error_msg)
        # Don't update current_fig on error to preserve last good plot
    finally:
        self._hide_loading()
        self._debug_log("=== Finished update_plot() ===")

LoggerDataViewer.update_plot = update_plot

In [ ]:
# Line graph plotting method (Part 1: data preparation and figure creation)
def _plot_line_graph(self):
    """Generate line graph plot."""
    self._debug_log("--- Starting _plot_line_graph() ---")
    
    # Map display names back to logger IDs
    selected_display_names = [l for l, cb in self.logger_checkboxes.items() if cb.value]
    name_to_id = {v: k for k, v in self.logger_display_names.items()}
    selected_loggers = [name_to_id[name] for name in selected_display_names]
    
    selected_metrics = [m.lower() for m, cb in self.metric_checkboxes.items() if cb.value]
    
    self._debug_log(f"Selected loggers: {selected_loggers}")
    self._debug_log(f"Selected metrics: {selected_metrics}")
    
    if not selected_loggers or not selected_metrics:
        self._debug_log("No loggers or metrics selected - returning None")
        # VERBOSE: print("Select at least one logger and one metric.")
        return None
    
    mode = self.time_mode.value
    
    selectors = {
        "selected_loggers": selected_loggers,
        "between_box": self.between_box,
        "year_select": self.year_select,
        "month_select": self.month_select,
        "week_select": self.week_select,
        "day_select": self.day_select
    }
    
    mask = get_time_mask(self.df, mode, selectors)
    subset = self.df[mask].copy()
    
    self._debug_log(f"Original data shape: {self.df.shape}")
    self._debug_log(f"Filtered data shape: {subset.shape}")
    
    if subset.empty:
        self._debug_log("Filtered data is empty - returning None")
        # VERBOSE: print("No data for selected filters.")
        return None
    
    self._debug_log(f"Data shape before gap processing: {subset.shape}")
    
    # Create dynamic title based on selected metrics
    if len(selected_metrics) == 2:  # Both temperature and humidity
        title_suffix = "Temperature & Humidity"
    elif "temperature" in selected_metrics:  # Only temperature
        title_suffix = "Temperature"
    else:  # Only humidity
        title_suffix = "Humidity"
    
    title = f"<b>Omnisense Monitoring – {title_suffix}</b>"
    
    # Create empty figure with title
    fig = go.Figure()
    fig.update_layout(title=title)
    
    # Line thickness proportional to "complexity" (time span proxy)
    if mode == "All time":
        line_width = 1.0
    elif mode in ("Between dates", "Year", "Month", "Week"):
        line_width = 1.4
    elif mode == "Day":
        line_width = 2.2
    else:
        line_width = 1.6
    
    # Create gap-broken traces for each logger
    self._debug_log("Creating gap-broken traces...")
    traces = create_gap_broken_traces(
        subset, 
        selected_metrics, 
        self.logger_display_names, 
        self.colour_map, 
        max_gap_hours=12.0
    )
    
    # Add each trace to the figure
    for i, trace in enumerate(traces):
        self._debug_log(f"Adding trace {i}: Logger={trace.legendgroup}, Points={len(trace.x)}, Name='{trace.name}', ShowLegend={trace.showlegend}")
        trace.line.width = line_width
        trace.opacity = 0.85
        fig.add_trace(trace)
    
    # Store for use in part 2
    return self._plot_line_graph_part2(fig, subset, selected_metrics, selected_loggers, mode)

LoggerDataViewer._plot_line_graph = _plot_line_graph

In [ ]:
# Line graph plotting method (Part 2: axes, formatting, and decorations)
def _plot_line_graph_part2(self, fig, subset, selected_metrics, selected_loggers, mode):
    """Complete line graph with axes, threshold, and season lines."""
    
    # Update axis labels with custom formatting based on selected metrics
    if len(selected_metrics) == 2:  # Both temperature and humidity
        y_title = "Temperature/Humidity"
        y_suffix = "°C/%RH"
    elif "temperature" in selected_metrics:  # Only temperature
        y_title = "Temperature"
        y_suffix = "°C"
    else:  # Only humidity
        y_title = "Humidity"
        y_suffix = "%RH"
    
    fig.update_yaxes(
        title_text=y_title,
        tickformat="",  # Let plotly choose appropriate decimal places
        ticksuffix=y_suffix
    )
    
    # Custom x-axis formatting based on time mode
    if mode == "Day":
        # Show hours and minutes for single day
        fig.update_xaxes(
            title_text="Time",
            tickformat="%H:%M",
            dtick=3600000  # 1 hour intervals
        )
    elif mode == "Week":
        # Show day and time for week view
        fig.update_xaxes(
            title_text="Date/Time",
            tickformat="%a %d<br>%H:%M",
            dtick=21600000  # 6 hour intervals
        )
    elif mode == "Month":
        # Show date and time for month view
        fig.update_xaxes(
            title_text="Date/Time",
            tickformat="%d %b<br>%H:%M",
            dtick=86400000  # 1 day intervals
        )
    elif mode == "Year":
        # Show month and day for year view
        fig.update_xaxes(
            title_text="Date",
            tickformat="%b %d",
            dtick="M1"  # Monthly intervals
        )
    elif mode == "Between dates":
        # Determine appropriate format based on date range
        date_range = (subset.index.max() - subset.index.min()).days
        if date_range <= 1:
            # Less than 1 day - show hours
            fig.update_xaxes(
                title_text="Time",
                tickformat="%H:%M",
                dtick=3600000  # 1 hour intervals
            )
        elif date_range <= 7:
            # Less than 1 week - show day and time
            fig.update_xaxes(
                title_text="Date/Time",
                tickformat="%a %d<br>%H:%M",
                dtick=21600000  # 6 hour intervals
            )
        elif date_range <= 31:
            # Less than 1 month - show date and time
            fig.update_xaxes(
                title_text="Date/Time",
                tickformat="%d %b<br>%H:%M",
                dtick=86400000  # 1 day intervals
            )
        elif date_range <= 365:
            # Less than 1 year - show month and day
            fig.update_xaxes(
                title_text="Date",
                tickformat="%b %d",
                dtick="M1"  # Monthly intervals
            )
        else:
            # More than 1 year - show year and month
            fig.update_xaxes(
                title_text="Date",
                tickformat="%Y<br>%b",
                dtick="M3"  # Quarterly intervals
            )
    else:
        # All time - adaptive based on total data range
        total_range = (self.df.index.max() - self.df.index.min()).days
        if total_range <= 31:
            fig.update_xaxes(
                title_text="Date/Time",
                tickformat="%d %b<br>%H:%M",
                dtick=86400000  # 1 day intervals
            )
        elif total_range <= 365:
            fig.update_xaxes(
                title_text="Date",
                tickformat="%b %d",
                dtick="M1"  # Monthly intervals
            )
        else:
            fig.update_xaxes(
                title_text="Date",
                tickformat="%Y<br>%b",
                dtick="M3"  # Quarterly intervals
            )
    
    fig.update_layout(
        height=700,
        width=950,
        margin=dict(l=80, r=40, t=35, b=80),
        template="plotly_white",
        legend=dict(orientation="h", y=-0.15, title=""),
        hovermode="closest",  # Remove the moving vertical line
        title_x=0.5,  # Center the title
        plot_bgcolor="white",
        paper_bgcolor="white"
    )
    
    return self._plot_line_graph_part3(fig, subset, selected_metrics, selected_loggers)

LoggerDataViewer._plot_line_graph_part2 = _plot_line_graph_part2

In [ ]:
# Line graph plotting method (Part 3: legend, threshold, and season lines)
def _plot_line_graph_part3(self, fig, subset, selected_metrics, selected_loggers):
    """Add legend ordering, threshold line, and season lines."""
    
    # Force legend order by reordering traces
    desired_order = [self.logger_display_names[logger_id] for logger_id in self.unique_loggers if logger_id in selected_loggers]
    
    # Get current traces and reorder them
    current_traces = list(fig.data)
    reordered_traces = []
    
    # Add traces in desired order (group by legendgroup/logger)
    for desired_name in desired_order:
        for trace in current_traces:
            if (hasattr(trace, 'legendgroup') and trace.legendgroup == desired_name) or \
               (hasattr(trace, 'name') and trace.name == desired_name):
                if trace not in reordered_traces:
                    reordered_traces.append(trace)
    
    # Add any remaining traces (like threshold line, season lines)
    for trace in current_traces:
        if trace not in reordered_traces:
            reordered_traces.append(trace)
    
    # Update figure with reordered traces
    self._debug_log(f"Reordered {len(reordered_traces)} traces")
    fig.data = reordered_traces
    
    # Add 32°C threshold line if enabled (after main traces so it appears on top)
    if self.threshold_checkbox.value:
        # Create multiple points along the line for better hover coverage
        x_points = pd.date_range(subset.index.min(), subset.index.max(), periods=100).floor('s')
        y_points = [32] * len(x_points)
        fig.add_scatter(
            x=x_points,
            y=y_points,
            mode='lines',
            line=dict(color="green", width=2, dash="solid"),
            name="32°C Threshold",
            hovertemplate="32°C Threshold<br>%{x|%d/%m/%Y %H:%M}<br>Temperature: 32.00°C<extra></extra>",
            showlegend=True
        )
    
    # Add season rectangles with labels if enabled (after main traces so they appear on top)
    if self.season_lines_checkbox.value:
        start_date = subset.index.min()
        end_date = subset.index.max()
        season_lines = get_season_lines(start_date, end_date)
        
        if len(season_lines) > 0:
            y_max = subset[selected_metrics].max().max()
            y_min = subset[selected_metrics].min().min()
            
            # Calculate periods and check for overlaps
            periods = []
            centers = []
            for i in range(len(season_lines)):
                sdate = season_lines[i][0].tz_convert(None).to_pydatetime()
                if i < len(season_lines) - 1:
                    next_sdate = season_lines[i + 1][0].tz_convert(None).to_pydatetime()
                else:
                    next_sdate = end_date.tz_convert(None).to_pydatetime()
                period_days = (next_sdate - sdate).days
                periods.append(period_days)
                centers.append(sdate + (next_sdate - sdate) / 2)
            
            # Check if labels would overlap based on time range and label count
            total_time_span = (end_date - start_date).days
            num_labels = len(season_lines)
            
            # More sophisticated overlap detection
            if num_labels == 0:
                show_labels_directly = True
            elif total_time_span > 365:  # More than a year
                show_labels_directly = num_labels <= 8
            elif total_time_span > 180:  # More than 6 months
                show_labels_directly = num_labels <= 6
            elif total_time_span > 90:   # More than 3 months
                show_labels_directly = num_labels <= 4
            else:  # Less than 3 months
                show_labels_directly = num_labels <= 3
            
            # Create extended season rectangles that cover full data range
            data_start = start_date.tz_convert(None).to_pydatetime()
            data_end = end_date.tz_convert(None).to_pydatetime()
            
            # Add season for the very beginning if no season line exists there
            all_dates = [data_start] + [s[0].tz_convert(None).to_pydatetime() for s in season_lines] + [data_end]
            all_seasons = [get_season_for_date(data_start)] + [s[1] for s in season_lines] + [get_season_for_date(data_end)]
            
            # Create rectangles for each period
            for i in range(len(all_dates) - 1):
                rect_start = all_dates[i]
                rect_end = all_dates[i + 1]
                season_name = all_seasons[i]
                
                # Add vertical lines only for actual season boundaries (not data boundaries)
                if i > 0:  # Don't add line at very start
                    fig.add_shape(
                        type="line",
                        x0=rect_start, x1=rect_start,
                        y0=y_min, y1=y_max * 1.05,
                        line=dict(color="black", width=1, dash="dot"),
                        opacity=0.7
                    )
                
                # Add horizontal line at top
                fig.add_shape(
                    type="line",
                    x0=rect_start, x1=rect_end,
                    y0=y_max * 1.05, y1=y_max * 1.05,
                    line=dict(color="black", width=1, dash="dot")
                )
                
                # Add horizontal line at bottom
                fig.add_shape(
                    type="line",
                    x0=rect_start, x1=rect_end,
                    y0=y_min, y1=y_min,
                    line=dict(color="black", width=1, dash="dot"),
                    opacity=0.7
                )
                
                if show_labels_directly:
                    # Show labels directly if no overlap
                    min_period = min(periods) if periods else 30
                    font_size = 8 if min_period > 30 else 6
                    fig.add_annotation(
                        x=rect_start + (rect_end - rect_start) / 2,
                        y=y_max * 1.08,
                        text=season_name,
                        showarrow=False,
                        font=dict(size=font_size, color="black"),
                        bgcolor="rgba(255, 255, 255, 0.8)",
                        bordercolor="black",
                        borderwidth=1
                    )
                else:
                    # Add small hover button in center of season box
                    center_x = rect_start + (rect_end - rect_start) / 2
                    fig.add_scatter(
                        x=[center_x],
                        y=[y_max * 1.06],
                        mode='markers',
                        marker=dict(
                            size=8,
                            color="rgba(100, 100, 100, 0.8)",
                            symbol="circle",
                            line=dict(color="black", width=1)
                        ),
                        hovertext=season_name,
                        hoverinfo='text',
                        showlegend=False,
                        name=f"Season: {season_name}"
                    )
    
    self._debug_log("--- Finished _plot_line_graph() successfully ---")
    return fig

LoggerDataViewer._plot_line_graph_part3 = _plot_line_graph_part3

In [ ]:
# Adaptive comfort plotting methods
def _get_boundary_params(self):
    """Return (m, c, delta) for the selected comfort model."""
    model = self.boundary_model.value
    if model == "default":
        return 0.31, 17.3, 3.0
    elif model == "rh_gt_60":
        return 0.53, 12.85, 2.84
    elif model == "rh_40_60":
        return 0.53, 14.16, 3.70
    elif model == "rh_le_40":
        return 0.52, 15.23, 4.40
    elif model == "none":
        return None
    return None

def _add_comfort_band(self, fig, ac_data):
    """Add comfort band to the figure."""
    params = self._get_boundary_params()
    if params is None:
        return
    
    m, c, delta = params
    
    x_min = ac_data["external_temp"].min()
    x_max = ac_data["external_temp"].max()
    
    if pd.isna(x_min) or pd.isna(x_max) or x_min == x_max:
        return
    
    x_vals = np.linspace(x_min, x_max, 80)
    y_central = m * x_vals + c
    y_upper = y_central + delta
    y_lower = y_central - delta
    
    # Store comfort boundaries for statistics calculation
    self._comfort_boundaries = {
        'x_vals': x_vals,
        'y_upper': y_upper,
        'y_lower': y_lower,
        'm': m, 'c': c, 'delta': delta
    }
    
    # Green comfort band only
    x_band = np.concatenate([x_vals, x_vals[::-1]])
    y_band = np.concatenate([y_lower, y_upper[::-1]])
    fig.add_trace(
        go.Scatter(
            x=x_band,
            y=y_band,
            fill="toself",
            mode="lines",
            line=dict(width=0),
            fillcolor="rgba(0, 150, 0, 0.3)",
            hoverinfo="skip",
            showlegend=False,
            name="Comfort Zone"
        )
    )

def _calculate_comfort_percentage(self, data, logger_id):
    """Calculate percentage of points within comfort zone for a logger."""
    if not hasattr(self, '_comfort_boundaries'):
        return 0.0
    
    logger_data = data[data['logger_id'] == logger_id]
    if logger_data.empty:
        return 0.0
    
    boundaries = self._comfort_boundaries
    m, c, delta = boundaries['m'], boundaries['c'], boundaries['delta']
    
    # Calculate comfort boundaries for each point
    ext_temps = logger_data['external_temp'].values
    temps = logger_data['temperature'].values
    
    y_upper = m * ext_temps + c + delta
    y_lower = m * ext_temps + c - delta
    
    # Count points within comfort zone
    within_comfort = (temps >= y_lower) & (temps <= y_upper)
    percentage = (within_comfort.sum() / len(temps)) * 100
    
    return percentage

def _update_comfort_stats(self, ac_data, selected_room_loggers):
    """Update comfort statistics display."""
    self.comfort_stats_out.clear_output()
    
    with self.comfort_stats_out:
        # Calculate percentages for each selected room
        room_percentages = {}
        total_points = 0
        total_comfort_points = 0
        
        for logger_id in selected_room_loggers:
            percentage = self._calculate_comfort_percentage(ac_data, logger_id)
            room_percentages[logger_id] = percentage
            
            logger_data = ac_data[ac_data['logger_id'] == logger_id]
            if not logger_data.empty:
                total_points += len(logger_data)
                total_comfort_points += int((percentage / 100) * len(logger_data))
        
        # Calculate overall percentage
        overall_percentage = (total_comfort_points / total_points * 100) if total_points > 0 else 0
        
        # Create HTML display
        html_content = f"""
        <div style="display: flex; flex-wrap: wrap; gap: 5px; max-width: 380px;">
            <div style="width: 100%; text-align: center; padding: 6px; border: 2px solid #333; background-color: #f0f0f0; border-radius: 3px; margin-bottom: 5px; font-size: 12px;">
                <strong>Overall: {overall_percentage:.1f}%</strong>
            </div>
        """
        
        # Add individual room boxes (up to 6, arranged in 2 rows of 3)
        for i, logger_id in enumerate(selected_room_loggers[:6]):
            room_name = self.logger_display_names[logger_id].split(' (')[0]  # Just room name without ID
            percentage = room_percentages[logger_id]
            
            html_content += f"""
            <div style="width: 90px; height: 45px; padding: 3px; border: 1px solid #666; background-color: #fff; border-radius: 3px; text-align: center; font-size: 10px; display: flex; flex-direction: column; justify-content: center;">
                <div style="font-weight: bold; line-height: 1.1; margin-bottom: 2px;">{room_name}</div>
                <div style="color: #007700; font-weight: bold; font-size: 11px;">{percentage:.1f}%</div>
            </div>
            """
        
        html_content += "</div>"
        
        display(widgets.HTML(html_content))

LoggerDataViewer._get_boundary_params = _get_boundary_params
LoggerDataViewer._add_comfort_band = _add_comfort_band
LoggerDataViewer._calculate_comfort_percentage = _calculate_comfort_percentage
LoggerDataViewer._update_comfort_stats = _update_comfort_stats

In [ ]:
# Adaptive comfort main plotting method
def _plot_adaptive_comfort(self):
    """Generate adaptive comfort scatter plot."""
    self._debug_log("--- Starting _plot_adaptive_comfort() ---")
    mode = self.time_mode.value
    
    # Get selected room loggers from checkboxes
    selected_room_display_names = [l for l, cb in self.room_logger_checkboxes.items() if cb.value]
    name_to_id = {v: k for k, v in self.logger_display_names.items()}
    selected_room_loggers = [name_to_id[name] for name in selected_room_display_names if name in name_to_id]
    
    self._debug_log(f"Selected room loggers: {selected_room_loggers}")
    
    if not selected_room_loggers:
        self._debug_log("No room loggers selected - returning None")
        # VERBOSE: print("Select at least one room logger for adaptive comfort chart.")
        return None
    
    # Create time mask for selected room loggers + external logger (if exists)
    loggers_for_mask = selected_room_loggers[:]
    if self.external_logger:
        loggers_for_mask.append(self.external_logger)
    
    selectors = {
        "selected_loggers": loggers_for_mask,
        "between_box": self.between_box,
        "year_select": self.year_select,
        "month_select": self.month_select,
        "week_select": self.week_select,
        "day_select": self.day_select
    }
    
    mask = get_time_mask(self.df, mode, selectors)
    
    # Prepare adaptive comfort data
    ac_data = prepare_adaptive_comfort_data(self.df, mask, self.external_logger, self.room_loggers)
    
    self._debug_log(f"Adaptive comfort data shape: {ac_data.shape}")
    
    if ac_data.empty:
        self._debug_log("No adaptive comfort data available - returning None")
        # VERBOSE: print("No data available for adaptive comfort chart in selected time range.")
        return None
    
    # Filter for only selected room loggers
    ac_data = ac_data[ac_data["logger_id"].isin(selected_room_loggers)].copy()
    
    self._debug_log(f"Filtered adaptive comfort data shape: {ac_data.shape}")
    
    if ac_data.empty:
        self._debug_log("No data for selected room loggers - returning None")
        # VERBOSE: print("No data available for selected room loggers in the time range.")
        return None
    
    # Add datetime column for hover
    ac_data["datetime"] = ac_data.index
    
    # Map logger IDs to display names
    ac_data["logger_display"] = ac_data["logger_id"].map(self.logger_display_names)
    
    # Create title with credit if using Vellei et al. models
    title = f"<b>Adaptive Comfort Chart – {mode}</b>"
    if self.boundary_model.value in ["rh_gt_60", "rh_40_60", "rh_le_40"]:
        title += "<br><span style='font-size:12px; line-height:0.8'>Boundary source: Vellei et al.</span>"
    
    # Create scatter plot
    fig = px.scatter(
        ac_data,
        x="external_temp",
        y="temperature",
        color="logger_display",
        title=title,
        labels={
            "external_temp": "Running mean external temperature (°C)",
            "temperature": "Air temperature (approximation of operative temperature)",
            "logger_display": "Room logger",
            "datetime": "Date/time"
        },
        color_discrete_map={self.logger_display_names[k]: v for k, v in self.colour_map.items()},
        opacity=0.6,
        hover_data={"datetime": "|%d/%m/%Y %H:%M"},
        category_orders={"logger_display": [self.logger_display_names[logger_id] for logger_id in selected_room_loggers]}
    )
    
    fig.update_layout(
        height=700,
        width=950,
        margin=dict(l=80, r=40, t=25, b=80),
        template="plotly_white",
        legend=dict(orientation="h", y=-0.15, title=""),
        hovermode="closest",
        title_x=0.5,  # Center the title
        plot_bgcolor="white",
        paper_bgcolor="white"
    )
    
    # Force legend order for adaptive comfort by reordering traces
    room_loggers_in_data = [logger_id for logger_id in selected_room_loggers if logger_id in ac_data["logger_id"].unique()]
    desired_order = [self.logger_display_names[logger_id] for logger_id in room_loggers_in_data]
    
    # Get current traces and reorder them
    current_traces = list(fig.data)
    reordered_traces = []
    
    # Add traces in desired order
    for desired_name in desired_order:
        for trace in current_traces:
            if hasattr(trace, 'name') and trace.name == desired_name:
                reordered_traces.append(trace)
                break
    
    # Add any remaining traces (like comfort band)
    for trace in current_traces:
        if trace not in reordered_traces:
            reordered_traces.append(trace)
    
    # Update figure with reordered traces
    fig.data = reordered_traces
    
    fig.update_traces(marker=dict(size=3), opacity=0.85)
    
    # Add comfort band and above/below shading if selected
    if self.boundary_model.value != "none":
        self._add_comfort_band(fig, ac_data)
        self._update_comfort_stats(ac_data, selected_room_loggers)
    else:
        self.comfort_stats_out.clear_output()
    
    self._debug_log("--- Finished _plot_adaptive_comfort() successfully ---")
    return fig

LoggerDataViewer._plot_adaptive_comfort = _plot_adaptive_comfort

In [ ]:
# Download methods (Part 1: simple chart download)
def _download_chart(self, button):
    """Download the current chart as an image."""
    if hasattr(self, 'current_fig') and self.current_fig is not None:
        try:
            # Create images folder if it doesn't exist
            import os
            images_dir = Path("images")
            images_dir.mkdir(exist_ok=True)
            
            # Create a filename with timestamp
            timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = images_dir / f"omnisense_chart_{timestamp}.png"
            
            # Create a copy of the figure with extra top margin for white space
            fig_copy = go.Figure(self.current_fig)
            current_margin = fig_copy.layout.margin
            fig_copy.update_layout(
                margin=dict(
                    l=current_margin.l,
                    r=current_margin.r,
                    t=current_margin.t + 40,  # Add 40px white space above title
                    b=current_margin.b
                )
            )
            
            # Download the figure
            fig_copy.write_image(filename, width=1200, height=840, scale=2)
            # VERBOSE: print(f"Chart saved as {filename.absolute()}")
        except Exception as e:
            pass
            # VERBOSE: print(f"Error saving chart: {e}")
            # VERBOSE: print("Make sure kaleido is installed: pip install kaleido")
    else:
        pass
        # VERBOSE: print("No chart to download. Please generate a chart first.")

LoggerDataViewer._download_chart = _download_chart

In [ ]:
# Download with statistics and display methods
def _download_chart_with_stats(self, button):
    """Download the adaptive comfort chart with statistics boxes."""
    if hasattr(self, 'current_fig') and self.current_fig is not None and self.chart_type.value == "Adaptive Comfort":
        try:
            # Create images folder if it doesn't exist
            import os
            images_dir = Path("images")
            images_dir.mkdir(exist_ok=True)
            
            # Create a filename with timestamp
            timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = images_dir / f"omnisense_adaptive_with_stats_{timestamp}.png"
            
            # Create a combined image using matplotlib
            import matplotlib.pyplot as plt
            import matplotlib.patches as patches
            from matplotlib.patches import FancyBboxPatch
            import io
            from PIL import Image
            
            # Export the plotly figure to image bytes
            fig_bytes = self.current_fig.to_image(format="png", width=1200, height=700, scale=2)
            chart_img = Image.open(io.BytesIO(fig_bytes))
            
            # Create matplotlib figure for combined layout
            fig, (ax_stats, ax_chart) = plt.subplots(2, 1, figsize=(15, 12), 
                                                    gridspec_kw={'height_ratios': [1, 4], 'hspace': 0.05})
            
            # Hide axes
            ax_stats.axis('off')
            ax_chart.axis('off')
            
            # Add white space at top
            fig.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95)
            
            # Get comfort statistics data
            if hasattr(self, '_comfort_boundaries'):
                selected_room_display_names = [l for l, cb in self.room_logger_checkboxes.items() if cb.value]
                name_to_id = {v: k for k, v in self.logger_display_names.items()}
                selected_room_loggers = [name_to_id[name] for name in selected_room_display_names if name in name_to_id]
                
                # Calculate statistics (reuse existing logic)
                loggers_for_mask = selected_room_loggers[:]
                if self.external_logger:
                    loggers_for_mask.append(self.external_logger)
                
                selectors = {
                    "selected_loggers": loggers_for_mask,
                    "between_box": self.between_box,
                    "year_select": self.year_select,
                    "month_select": self.month_select,
                    "week_select": self.week_select,
                    "day_select": self.day_select
                }
                mask = get_time_mask(self.df, self.time_mode.value, selectors)
                ac_data = prepare_adaptive_comfort_data(self.df, mask, self.external_logger, self.room_loggers)
                ac_data = ac_data[ac_data["logger_id"].isin(selected_room_loggers)].copy()
                
                # Calculate percentages
                room_percentages = {}
                total_points = 0
                total_comfort_points = 0
                
                for logger_id in selected_room_loggers:
                    percentage = self._calculate_comfort_percentage(ac_data, logger_id)
                    room_percentages[logger_id] = percentage
                    
                    logger_data = ac_data[ac_data['logger_id'] == logger_id]
                    if not logger_data.empty:
                        total_points += len(logger_data)
                        total_comfort_points += int((percentage / 100) * len(logger_data))
                
                overall_percentage = (total_comfort_points / total_points * 100) if total_points > 0 else 0
                
                # Draw statistics boxes
                ax_stats.text(0.5, 0.8, "Comfort Statistics", ha='center', va='center', 
                            fontsize=16, fontweight='bold', transform=ax_stats.transAxes)
                
                # Overall box
                overall_box = FancyBboxPatch((0.35, 0.5), 0.3, 0.15, 
                                           boxstyle="round,pad=0.01", 
                                           facecolor='#f0f0f0', edgecolor='black', linewidth=2)
                ax_stats.add_patch(overall_box)
                ax_stats.text(0.5, 0.575, f"Overall: {overall_percentage:.1f}%", 
                            ha='center', va='center', fontsize=12, fontweight='bold',
                            transform=ax_stats.transAxes)
                
                # Individual room boxes
                box_width = 0.12
                box_height = 0.12
                start_x = 0.1
                y_pos = 0.25
                
                for i, logger_id in enumerate(selected_room_loggers[:6]):
                    x_pos = start_x + (i % 3) * (box_width + 0.05)
                    if i >= 3:
                        y_pos = 0.1
                        x_pos = start_x + ((i - 3) % 3) * (box_width + 0.05)
                    
                    room_name = self.logger_display_names[logger_id].split(' (')[0]
                    percentage = room_percentages[logger_id]
                    
                    room_box = FancyBboxPatch((x_pos, y_pos), box_width, box_height,
                                            boxstyle="round,pad=0.005",
                                            facecolor='white', edgecolor='#666', linewidth=1)
                    ax_stats.add_patch(room_box)
                    
                    ax_stats.text(x_pos + box_width/2, y_pos + box_height*0.7, room_name,
                                ha='center', va='center', fontsize=9, fontweight='bold',
                                transform=ax_stats.transAxes)
                    ax_stats.text(x_pos + box_width/2, y_pos + box_height*0.3, f"{percentage:.1f}%",
                                ha='center', va='center', fontsize=10, fontweight='bold', color='#007700',
                                transform=ax_stats.transAxes)
            
            # Add the chart image
            ax_chart.imshow(chart_img)
            
            # Save the combined figure
            plt.savefig(filename, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close()
            
            # VERBOSE: print(f"Chart with statistics saved as {filename.absolute()}")
            
        except Exception as e:
            pass
            # VERBOSE: print(f"Error saving chart with statistics: {e}")
            # VERBOSE: print("Make sure kaleido, matplotlib, and PIL are installed")
    else:
        pass
        # VERBOSE: print("No adaptive comfort chart to download. Please generate an adaptive comfort chart first.")

def display_viewer(self):
    """Display the viewer interface."""
    # Use module-level reference to avoid import issues in Voilà
    self._debug_log("Initializing viewer interface...")
    _ipython_display(self.time_controls, self.controls, self.plot_out)
    self._debug_log("Interface displayed, updating initial plot...")
    self.update_plot()
    self._debug_log("Initial setup complete!")

LoggerDataViewer._download_chart_with_stats = _download_chart_with_stats
LoggerDataViewer.display = display_viewer

In [7]:
viewer = LoggerDataViewer()
viewer.display()

Found 1 Omnisense CSV file(s) in data/omnisense

Processing 070226.csv...
Loaded 2501 records from: House 5, Kitchen
Loaded 2416 records from: House 5, Bed 4
Loaded 2498 records from: House 5, Bed 4, above ceiling
Loaded 2500 records from: House 5, Bed 2
Loaded 2499 records from: House 5, Living Room
Loaded 2498 records from: House 5, Mother's Bedroom
Loaded 2499 records from: House 5, Washrooms area
Loaded 2500 records from: House 3, Bed 2
Loaded 2498 records from: House 5, Bed 3
Skipping Sun, Wind, Rain weather station gateway (in external box) (no temperature/humidity data)
Skipping Weather Station, T & RH (weather station)
Loaded 4583 records from: House 5, Metal Roof, above Bed 4
Skipping Performance stats (no temperature/humidity data)

Loading external temperature data...
Loaded 504 external temperature records from Open-Meteo
  Date range: 2026-01-17 00:00:00 to 2026-02-06 23:00:00

Total datasets loaded: 2
Total records: 27496
Unique loggers: 11
Logger IDs: ['External (Open-Me

Output()

In [ ]:
# Example of EN15251 running mean calculation working correctly:
# 
# Sample data from external logger (861011) for first few days:
# Day 1: 25.5°C daily mean -> Trm[0] = 25.5°C (seed value)
# Day 2: 26.2°C daily mean -> Trm[1] = (1-0.8)*26.2 + 0.8*25.5 = 0.2*26.2 + 0.8*25.5 = 5.24 + 20.4 = 25.64°C
# Day 3: 24.8°C daily mean -> Trm[2] = (1-0.8)*24.8 + 0.8*25.64 = 0.2*24.8 + 0.8*25.64 = 4.96 + 20.512 = 25.472°C
# Day 4: 27.1°C daily mean -> Trm[3] = (1-0.8)*27.1 + 0.8*25.472 = 0.2*27.1 + 0.8*25.472 = 5.42 + 20.3776 = 25.7976°C
#
# This exponential weighting gives more influence to recent temperatures while maintaining
# a smooth running average that responds gradually to temperature changes, as required by EN15251.